In [11]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(4):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/calculating_scores/DFL/FULL/fedstellar_DFL_18_07_2024_11_06_34/metrics/participant_{i}"
    date_str = "18/07/2024 9:06:34	" # minus two hours for timezone

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step     value                     timestamp         tag
0   201  0.287095 2024-07-18 09:09:13.856707840  Train/Loss
1   251  0.267699 2024-07-18 09:09:14.404602624  Train/Loss
2   301  0.212940 2024-07-18 09:09:14.954572288  Train/Loss
3   351  0.155932 2024-07-18 09:09:15.513546496  Train/Loss
4   401  0.221429 2024-07-18 09:09:16.061843200  Train/Loss
2024-07-18 09:22:41 2024-07-18 09:06:44
                 step     value                     timestamp
tag                                                          
Test/Accuracy   11042  0.886368 2024-07-18 09:22:41.049877760
Test/F1Score    11042  0.864669 2024-07-18 09:22:41.049907968
Test/Loss       11042  0.438543 2024-07-18 09:22:41.049836288
Test/Precision  11042  0.880390 2024-07-18 09:22:41.049888768
Test/Recall     11042  0.886368 2024-07-18 09:22:41.049898496
   step     value                     timestamp         tag
0   200  0.193333 2024-07-18 09:09:13.807701248  Train/Loss
1   250  0.162730 2024-07-18 09:09:14.35743436

In [12]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,11042,0.886368,2024-07-18 09:22:41.049877760,participant_0
Test/F1Score,11042,0.864669,2024-07-18 09:22:41.049907968,participant_0
Test/Loss,11042,0.438543,2024-07-18 09:22:41.049836288,participant_0
Test/Precision,11042,0.880390,2024-07-18 09:22:41.049888768,participant_0
Test/Recall,11042,0.886368,2024-07-18 09:22:41.049898496,participant_0
Test/Accuracy,5498,0.884178,2024-07-18 09:22:41.278440704,participant_1
Test/F1Score,5498,0.867407,2024-07-18 09:22:41.278471680,participant_1
Test/Loss,5498,0.421844,2024-07-18 09:22:41.278410240,participant_1
Test/Precision,5498,0.886065,2024-07-18 09:22:41.278452224,participant_1


In [13]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_33691/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,9690.25,0.8858
Test/F1Score,9690.25,0.8665
Test/Loss,9690.25,0.4304
Test/Precision,9690.25,0.8835
Test/Recall,9690.25,0.8858


In [14]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_33691/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,2795.02,0.00
Test/F1Score,2795.02,0.00
Test/Loss,2795.02,0.01
Test/Precision,2795.02,0.00
Test/Recall,2795.02,0.00


In [ ]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

In [4]:
concat_df.to_csv("results_csvs/dfl_15_ring.csv")